<a href="https://colab.research.google.com/github/Germandev55/Topology_and_Geometry-for-Deep-Learning/blob/main/ID_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D

import IPython
from IPython.display import HTML
from google.colab import output

import tensorflow as tf
from keras.datasets import cifar10
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, BatchNormalization, Dropout, Convolution2D, Input,Activation, ZeroPadding2D, MaxPooling2D, Flatten
from keras import regularizers

import sklearn
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn import manifold, datasets
from sklearn.manifold import TSNE
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance_matrix
from scipy.stats import pearsonr

min_max_scaler = preprocessing.MinMaxScaler()
max_abs_scaler = preprocessing.MaxAbsScaler()
scaler = StandardScaler()
  
!pip install ripser

import random 
from ripser import ripser
from ripser import Rips

import math
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d

In [2]:
%%capture
!pip install fermat
!pip install scikit-dimension
from fermat import Fermat
import skdim

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Conver and normalize images [0..255] --> [0..1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
# Standarize Images центрирование по каждому каналу 
mean = np.mean(x_train,axis=(0, 1, 2, 3))
std = np.std(x_train,axis=(0, 1, 2, 3)) # стандартное отклонение
x_train = (x_train-mean)/(std + 1e-7)
x_test = (x_test-mean)/(std + 1e-7)

170508288/170498071 [==============================] - 2s 0us/step


## Вспомогательные функции

In [4]:
def features_to_X(x_data, model, index_layer):
  input = x_data
  layer = model.get_layer(index=index_layer) 
  #print(layer)
  #layer_name = str(layer).split('.')[5].split(' ')[0]
  #layer_name = layer
  features_layer1 = tf.keras.models.Model(inputs=model.inputs, outputs = layer.output)
  output_ = features_layer1(input) 
  try:
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    #global_average_layer = tf.keras.layers.Flatten()
    #global_average_layer  = tf.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=None, 
    #padding="valid", data_format=None)

    feature_batch_average = global_average_layer(output_)
    X = feature_batch_average
    #X = np.reshape(X,(X.shape[0], -1))
  except:
    X = output_ 
  #X = features_layer1.predict(input)  
  return X, layer

def get_Betti(data, eps, maxdim=1):
  Betti_layer = []
  dgms = ripser(data, maxdim=maxdim, thresh=eps)['dgms']  
  inf_in_list = lambda x: 1 if np.isinf(x).any() else 0
  Betti_list = [0,0,0]
  for e, i in enumerate(dgms):
    for x in i: Betti_list[e] += inf_in_list(x)
  return sum(Betti_list)

def get_Betti_prob(data, eps, alpha, maxdim=2):
  dgms = ripser(data, maxdim=maxdim, thresh=eps)['dgms']  
  prev_max = 0
  list_Hom = []
  mean_Hom = []
  for Hom in dgms:
    B_Hom = 0
    for i in Hom:   
      if (np.isinf(i).any() == False):
        B_Hom += (((i[1]-i[0]))**alpha)*0.5
        prev_max = i[1]
      else:
        #в случае infinity прибавляем последнее время жизни
        continue
        B_Hom += prev_max 

    list_Hom.append(B_Hom)
    l = len(Hom)
    if len(Hom) == 0: l=1
    mean_Hom.append(B_Hom/l)

  mean_Hom =  [round(x,3) for x in mean_Hom]
  return list_Hom, mean_Hom

In [5]:
def compute_MGST(X, hom_dim) -> float:
  MGST, _ = get_Betti_prob(X, eps=np.inf, alpha=1, maxdim=hom_dim)
  lfts = round(MGST[hom_dim], 3)
  return lfts

def MGST_layers(model, examples, layers, hom_dim, fermat_alpha, n_class=1, mode = 'local') -> list:
  class_ = n_class
  MGST_layers_list = []
  for lr in layers:
    X_emb, layer_name = features_to_X(x_train[:examples], model, index_layer=lr) 
    print(layer_name, X_emb.shape)
    X_emb = min_max_scaler.fit_transform(X_emb)   
      
    if mode == 'local':
      X_emb_class = []
      for e, i in enumerate(y_train[:examples]):
        if i == class_: X_emb_class.append(X_emb[e])
      Xtest_class = np.array(X_emb_class)
    else:
      Xtest_class = np.array(X_emb)

    MGST_ = compute_MGST_Fermat(Xtest_class, hom_dim, fermat_alpha)
    #MGST_ = get_Betti(Xtest_class, eps=eps, maxdim=0)

    print(f'{lr}) {MGST_}')
    MGST_layers_list.append(MGST_)
  
  return MGST_layers_list

def compute_MGST_Fermat(X, hom_dim, fermat_alpha) -> float:
  distances_X = distance_matrix(X, X)
  f_exact = Fermat(fermat_alpha, path_method='FW') 
  f_exact.fit(np.matrix(distances_X))
  fermat_dist_exact = f_exact.get_distances()
  MGST, _ = get_Betti_prob(fermat_dist_exact, eps=np.inf, alpha=1, maxdim=hom_dim)
  lfts = round(MGST[hom_dim], 3)
  return lfts

In [6]:
def power_law(x, a, b):
    return a*np.power(x, b)

def PHdim(X, hom_dim, n, alpha=1):
  n = 500
  n_array = []
  MGST_array = []
  while X.shape[0] > n:
    MGST, _ = get_Betti_prob(X[:n,:], np.inf, alpha, hom_dim)
    MGST_array.append(MGST[hom_dim])

    #MGST = compute_MGST_Fermat(X[:n,:], 0, 1)
    #MGST_array.append(MGST)
    n_array.append(n)
    n += 450

  x = np.array(n_array).reshape((-1, 1))
  y = np.array(MGST_array)
  model = LinearRegression().fit(np.log(x), np.log(y))
  pars, pcov = curve_fit(power_law,  np.array(n_array),y)
  #y = b0+b1*x
  b1 = model.coef_[0]
  b0 = model.intercept_
  r_sq = model.score(x, y)
  betta = math.log((sum(MGST_array)/len(MGST_array)))/math.log(n_array[-1])
  #d = alpha/(1-betta) #(не работает)
  #d = -1/(b1-1)
  #d = -1/(pars[1]-1)
  d = 1/(1-b1)
  return round(d,2)
#log(MGST) = a*log(n) + b + eps, α = (m − γ)/m

In [11]:
def PHDim_layers(model, data, layers, hom_dim, alpha):
  class_ = 1
  examples = 500
  PHDim_layers_list = []
  #layers_check = layers
  SVDDim_layers_list = []
  for lr in layers:
    X_emb, layer_name = features_to_X(data, model, index_layer=lr) 
    #print(layer_name, X_emb.shape)
    #X_emb = min_max_scaler.fit_transform(X_emb)
    
    X_emb_class = []
    for e, i in enumerate(y_train[:examples]):
      if i == class_: X_emb_class.append(X_emb[e])
    X_emb_class = np.array(X_emb_class)

    X_emb_class = np.array(X_emb)
    PHdim_ = PHdim(X_emb_class, hom_dim = hom_dim, n = X_emb_class.shape[0], alpha=alpha)
  
    #print(f'{lr}) {PHdim_}')
    PHDim_layers_list.append(PHdim_)
  return PHDim_layers_list
  
def unistscount(layer_list: list) -> list:
  return [round((e+1)/len(layer_list),2) for e,i in enumerate(layer_list)]

def cocycles():
  ripser(x_train[:1000].reshape((1000, 32*32*3)), maxdim=1, thresh=np.inf, do_cocycles=True)['cocycles'] 

In [8]:
from math import sqrt
def estimate_ID(X, fraction=0.9):            
    # sort distance matrix
    Y = np.sort(X,axis=1,kind='quicksort')
    # clean data
    k1 = Y[:,1]
    k2 = Y[:,2]

    zeros = np.where(k1 == 0)[0]
    degeneracies = np.where(k1 == k2)[0]
    good = np.setdiff1d(np.arange(Y.shape[0]), np.array(zeros) )
    good = np.setdiff1d(good,np.array(degeneracies))

    k1 = k1[good]
    k2 = k2[good]    
    
    # n.of points to consider for the linear regression
    npoints = int(np.floor(good.shape[0]*fraction))

    # define mu and Femp
    N = good.shape[0]
    mu = np.sort(np.divide(k2, k1), axis=None,kind='quicksort')
    Femp = (np.arange(1,N+1,dtype=np.float64) )/N
    
    # take logs (leave out the last element because 1-Femp is zero there)
    x = np.log(mu[:-2])
    y = -np.log(1 - Femp[:-2])

    # regression
    regr = linear_model.LinearRegression(fit_intercept=False)
    regr.fit(x[0:npoints,np.newaxis],y[0:npoints,np.newaxis]) 
    r,pval = pearsonr(x[0:npoints], y[0:npoints])  
    return x,y,regr.coef_[0][0],r,pval

def estimate_ID_layers(model,data,layers_list,):
    list_ID = []
    for lr in layers_list:
      X_emb, layer_name = features_to_X(data, model, index_layer=lr) 
      distances_X = distance_matrix(X_emb, X_emb)
      ID = estimate_ID(distances_X, fraction=1.0)
      list_ID.append(round(ID[2], 2))
    return list_ID

# Эксперименты

In [12]:
# ИЗМЕНЕНИЕ PHdim ПО СЛОЯМ У МОДЕЛИ С РАЗНОЙ ШИРИНОЙ СЛОЁВ
%%capture
examples = 2000
PHdim_Resnet_6_96 = [22] + PHDim_layers(Resnet_6_96, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_64 = [22] + PHDim_layers(Resnet_6_64, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_32 = [22] + PHDim_layers(Resnet_6_32, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_20 = [22] + PHDim_layers(Resnet_6_20, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_256 = [22] + PHDim_layers(Resnet_6_256, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_128 = [22] + PHDim_layers(Resnet_6, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_352 = [22] + PHDim_layers(Resnet_6_352, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_412 = [22] + PHDim_layers(Resnet_6_412, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)
PHdim_Resnet_6_512 = [22] + PHDim_layers(Resnet_6_512, x_train[:examples], [14, 27, 49, 71, 93, 115, -2, -1], 0, alpha=1)



In [ ]:
PHdim(X_emb_class, hom_dim = hom_dim, n = X_emb_class.shape[0], alpha=alpha)

In [36]:
data_cifar10 = x_train[:examples].reshape((examples, 32*32*3))
PHdim_cifar10 = PHdim(data_cifar10, hom_dim = 0, n = data_cifar10.shape[0], alpha=1)
#distances_X = distance_matrix(data_cifar10, data_cifar10)
#ID = estimate_ID(distances_X, fraction=1.0)
print(round(PHdim_cifar10, 2))

22.81


In [57]:
%%capture
layers_check = [14, 27, 49, 71, 93, 115, -2, -1]
PHdim_cifar10 = [22]
examples = 2000
PHdim_Resnet_diff_width = []
for model in Resnet_diff_width:
  #PHdim_Resnet_diff_width.append(PHdim_cifar10 + PHDim_layers(model, x_train[:examples], layers_check, 0, alpha=1))
  PHdim_Resnet_diff_width.append(MGST_layers(model, examples, layers_check, 0, 1, n_class=2, mode = 'local'))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
diff_width_model_dim = PHdim_Resnet_diff_width
for ID_Resnet, name in zip(diff_width_model_dim, Resnet_diff_width_names):
  plt.plot(range(len(ID_Resnet)), ID_Resnet, 'o-',  linewidth=0.7, markersize=5, label=name)
ax.grid(which='major', color='#CCCCCC', linestyle='--', alpha=0.35)
plt.legend()
plt.title('ID_across_different width model')
plt.show()

In [ ]:
n = 3
ID_layer = [id_l[n] for id_l in ID_Resnet_diff_width ]
ID_max = [max(id_l[1:]) for id_l in PHdim_Resnet_diff_width]
depths = [20,32,64,96,128,256,352,412,512]
fig, ax = plt.subplots(figsize=(18, 8))
plt.plot(depths, ID_max, 'o-',  linewidth=0.7, markersize=5, label='depth vs id' )
ax.grid(which='major', color='#CCCCCC', linestyle='--', alpha=0.35)
ax.set_xticklabels(Resnet_diff_width_names,rotation=40)
plt.xticks([20,32,64,96,128,256,352,412,512])
plt.title('Фиксированная длина, разная ширина. Изменение максимальной внутренней фрактальной размерости в зависимости от ширины')
plt.show()

# Загрузка моделей

In [9]:
Resnet_6_128 = keras.models.load_model('/content/drive/MyDrive/ML_materials/resnet/Resnet_6_128_0.87.h5') 
Resnet_6_96 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet96_0.858.h5') 
Resnet_6_64 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_64_0.858.h5') 
Resnet_6_32 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_32_0.77.h5') 
Resnet_6_20 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_20_0.71.h5') 
Resnet_6_256 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_256_0.88.h5') 
Resnet_6_352 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_352_0.88.h5') 
Resnet_6_412 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_412_0.88.h5') 
Resnet_6_512 = keras.models.load_model('/content/drive/MyDrive/ML_materials/Resnet_type_96/Resnet_6_512_0.89.h5') 

In [20]:
Resnet_diff_width = [Resnet_6_20,Resnet_6_32,Resnet_6_64,Resnet_6_96,Resnet_6_128,Resnet_6_256,Resnet_6_352,Resnet_6_412,Resnet_6_512]
Resnet_diff_width_names = ['Resnet_6_20 71%','Resnet_6_32 77%','Resnet_6_64 85%','Resnet_6_96 86%', 'Resnet_6_128 87%', 'Resnet_6_256 88%',
                           'Resnet_6_352 88%','Resnet_6_412 88%','Resnet_6_512 89%']